In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_pickle('translated_tweets.pkl')

In [3]:
df.shape

(14923, 39)

In [8]:
df = df.drop_duplicates(subset='id')

In [10]:
df.columns

Index(['contributors', 'coordinates', 'created_at', 'entities',
       'favorite_count', 'favorited', 'filter_level', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'matching_rules',
       'place', 'quote_count', 'reply_count', 'retweet_count', 'retweeted',
       'source', 'text', 'truncated', 'user', 'possibly_sensitive',
       'display_text_range', 'extended_entities', 'extended_tweet',
       'quoted_status', 'quoted_status_id', 'quoted_status_id_str',
       'quoted_status_permalink', 'retweeted_status', 'created_at_date',
       'full_text', 'translated_text'],
      dtype='object')

In [11]:
df.loc[0]['entities']

{'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []}

In [ ]:
retweets = pd.DataFrame()
for d in df['retweeted_status'].dropna():
    tmp = pd.DataFrame.from_dict(d, orient='index').T
    retweets = retweets.append(tmp)

In [48]:
df[df['id'].isin(retweets['id'].drop_duplicates())].shape

(12, 39)

In [50]:
retweets.drop_duplicates(subset='id').shape

(12, 33)

In [52]:
quoted_tweets = pd.DataFrame()
for d in df['quoted_status'].dropna():
    tmp = pd.DataFrame.from_dict(d, orient='index').T
    quoted_tweets = quoted_tweets.append(tmp)

C:\Users\Hadrian\Anaconda3\envs\twitter-vaccine\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [53]:
quoted_tweets.shape

(735, 32)

In [54]:
quoted_tweets['id'].nunique()

584

In [55]:
df[df['id'].isin(quoted_tweets['id'].unique())].shape

(11, 39)

In [63]:
quoted_tweets['full_text'] = quoted_tweets['extended_tweet'].apply(lambda x: x.get('full_text') if type(x) == dict else None)

In [65]:
quoted_tweets['id'].value_counts()

1174582282268446720    55
1165454143726571521    12
1158743266436628480     8
1093158623239434240     7
1067712244874530816     5
                       ..
963285091916173312      1
715488194108592128      1
1093851038115713024     1
1057957954551312384     1
1093652991536652288     1
Name: id, Length: 584, dtype: int64

In [69]:
df = df.append(quoted_tweets.drop_duplicates(subset='id'))

In [74]:
df['created_at_date'] = pd.to_datetime(df['created_at'], utc=True)

In [75]:
df.to_pickle('2019-10-27_tweets_with_quotes.pkl')